In [142]:
import pandas as pd
import numpy as np
import warnings
import eventstudy as es
from tqdm import tqdm
import itertools
import pymannkendall as mk
import matplotlib.pyplot as plt 
import yfinance as yf
from scipy.stats import wilcoxon
from multiprocessing import cpu_count
from concurrent.futures import ThreadPoolExecutor
import os, sys
from pathlib import Path
import pickle
sys.path.append(str(Path(os.getcwd()).parent / '1226' / 'utils'))
import XD
from utils import process

warnings.filterwarnings("ignore")

In [164]:
q = pd.read_csv("../../豹投資/1223/data/quote/2020_20230814.csv", index_col=False).dropna()
brk = pd.read_csv("../../豹投資/1223/data/brk/2022_BrkNetAmt.csv", index_col=False).dropna()

In [165]:
df = pd.merge(q, brk, on=['日期', '股號'], how='left')
df = df.groupby('股號').apply(lambda x: x.sort_values('日期')).reset_index(drop=True)
df = df.rename(columns={'漲跌幅(%)':'ret'})

In [167]:
bk_uq_id = df['分點'].unique()

# Number of processes to use (adjust as needed)
num_processes = min(cpu_count(), len(bk_uq_id))

with ThreadPoolExecutor(max_workers=cpu_count()) as executor:
    list(tqdm(executor.map(process.process_bk, [bk for bk in bk_uq_id], [df for _ in range(len(bk_uq_id))]), total=len(bk_uq_id)))
    # list(executor.map(process.process_bk, [bk for bk in bk_uq_id], [df for _ in range(len(bk_uq_id))]), total=len(bk_uq_id))

 94%|█████████▎| 513/548 [07:02<00:28,  1.21it/s]


MemoryError: Unable to allocate 11.9 MiB for an array with shape (1561059,) and data type int64

In [153]:
df[df['股號']==1105]

,日期,股號,分點,買賣超金額,收盤價,ret,成交量,成交值(百萬),indicator_922H,indicator_116G,...,indicator_9287,indicator_9A61,indicator_9A9C,indicator_9884,indicator_981E,indicator_126i,indicator_9699,indicator_779G,indicator_116E,indicator_989g


In [137]:
# cols_index
query_cols=['ret','日期', '股號', 'indicator_116E']
cols_index = df.columns.get_indexer(query_cols)
print(df.columns)
print(cols_index)

Index(['日期', '股號', '分點', '買賣超金額', '收盤價', 'ret', '成交量', '成交值(百萬)',
       'indicator_9268', 'indicator_922H',
       ...
       'indicator_9324', 'indicator_983M', 'indicator_6462', 'indicator_116Z',
       'indicator_9A61', 'indicator_779G', 'indicator_981L', 'indicator_9235',
       'indicator_918A', 'indicator_116E'],
      dtype='object', length=121)
[  5   0   1 120]


In [138]:
np_df = df.to_numpy()
np_df_tick = np.array(np.split(np_df, np.unique(np_df[:, 1], return_index=True)[1][1:]))

In [139]:
vec_t = np.vectorize(process.transpose_element)
np_df_tick = vec_t(np_df_tick)

In [140]:
np_df_tick

array([array([['2022-02-09', '2022-03-15', '2022-03-18', ..., '2022-12-21',
               '2022-12-22', '2022-12-23'],
              [1101, 1101, 1101, ..., 1101, 1101, 1101],
              ['9268', '922H', '9268', ..., '9268', '9268', '9268'],
              ...,
              [0, 0, 0, ..., 0, 0, 0],
              [0, 0, 0, ..., 0, 0, 0],
              [0, 0, 0, ..., 0, 0, 0]], dtype=object)                      ,
       array([['2022-07-22', '2022-07-25', '2022-07-27', ..., '2022-12-21',
               '2022-12-22', '2022-12-23'],
              [1102, 1102, 1102, ..., 1102, 1102, 1102],
              ['922H', '779c', '592E', ..., '9268', '9268', '9268'],
              ...,
              [0, 0, 0, ..., 0, 0, 0],
              [0, 0, 0, ..., 0, 0, 0],
              [0, 0, 0, ..., 0, 0, 0]], dtype=object)                      ,
       array([['2022-02-18', '2022-02-22', '2022-02-24'],
              [1210, 1210, 1210],
              ['9202', '9202', '9202'],
              [184954642, 57

In [141]:
[arr[cols_index,:] for arr in np_df_tick]

[array([[0.75, 0.84, 1.98, -0.5, 0.1, -0.3, 0.24, 0.12, 1.09, 1.42, 1.28,
         0.76, -0.13, 2.15, 1.88, 2.33, 0.13, 1.92, 0.38, 0.25, -0.63,
         -0.25, -0.14, 6.22, 1.05, -2.17, 1.04, 1.17, -1.16],
        ['2022-02-09', '2022-03-15', '2022-03-18', '2022-03-25',
         '2022-03-31', '2022-04-01', '2022-05-18', '2022-05-20',
         '2022-05-25', '2022-05-30', '2022-06-24', '2022-06-27',
         '2022-06-29', '2022-07-01', '2022-07-05', '2022-07-19',
         '2022-08-31', '2022-09-06', '2022-09-08', '2022-09-13',
         '2022-09-14', '2022-09-15', '2022-09-27', '2022-11-14',
         '2022-11-25', '2022-12-19', '2022-12-21', '2022-12-22',
         '2022-12-23'],
        [1101, 1101, 1101, 1101, 1101, 1101, 1101, 1101, 1101, 1101, 1101,
         1101, 1101, 1101, 1101, 1101, 1101, 1101, 1101, 1101, 1101, 1101,
         1101, 1101, 1101, 1101, 1101, 1101, 1101],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]], dtype

In [134]:
len([0.75, 0.84, 1.98, -0.5, 0.1, -0.3, 0.24, 0.12, 1.09, 1.42, 1.28,
        0.76, -0.13, 2.15, 1.88, 2.33, 0.13, 1.92, 0.38, 0.25, -0.63,
        -0.25, -0.14, 6.22, 1.05, -2.17, 1.04, 1.17, -1.16])

29

In [136]:
len([1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 1, 1, 1, 1])

29

In [119]:
np.array([arr[[2,3],:] for arr in np_df_tick], dtype=object)

ValueError: could not broadcast input array from shape (2,29) into shape (2,)

In [71]:
import numpy as np

# Your original array
original_array = np.array([
    np.array([
        ['2022-02-09', '2022-03-15', '2022-03-18', '2022-12-21', '2022-12-22', '2022-12-23'],
        [1101, 1101, 1101, 1101, 1101, 1101],
        ['9268', '922H', '9268', '9604', '9268', '9268', '9268', '9268', '9268', '9268',
         '9268', '9268', '9268', '9268', '9268', '9268', '9268', '9268', '9268', '9268',
         '9268', '9268', '9268', '116G', '9623', '9268', '9268', '9268', '9268', '9268']
    ], dtype=object),
    np.array([
        ['2022-07-22', '2022-07-25', '2022-07-27', '2022-12-21', '2022-12-22', '2022-12-23'],
        [1102, 1102, 1102, 1102, 1102, 1102],
        ['922H', '779c', '592E', '9268', '9268', '9268', '9268', '9268', '9268']
    ], dtype=object)
], dtype=object)

# Rows to extract from each sub-array
rows_to_extract = [2, 1]  # Adjust as needed

# Create the desired array of arrays
result_array = np.array([arr[rows_to_extract, :] for arr in original_array], dtype=object)

# Print the result
print(result_array)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
stock_uq_id = df['股號'].unique()
g_df = df.groupby('股號')
res_d = {}
bug_li = []

for bk in tqdm(bk_uq_id[:]):
    
    res_tick = {}
    for tick in tqdm(stock_uq_id[:]):
        tmp = g_df.get_group(tick)
        try:
            # calculate cumulative event related return 
            c, cp = XD.get_indcum(col_ret = tmp['ret'], col_abnormal=tmp[f'indicator_{bk}'], num_period=N_P)
            tmp[f'cumret_{bk}_{N_P}'] = c
            
            # perform test
            cp = list(k for k,_ in itertools.groupby(cp))
            result_trend = {}
            
            # date list
            date_list = []
            
            for i, l in enumerate(cp):
                if len(l)>1:
                    if sum(l) == 0:
                        continue
                    else:
                        trend, h, p, z, Tau, s, var_s, slope, intercept =  mk.original_test(l)
                        result_trend[f'{i}_trend'] = trend
                        
                        # date index
                        date_index = tmp[tmp[f'indicator_{bk}']!=0].index[i]
                        date_list.append(tmp['日期'][date_index])
            

            if len(result_trend)!=0:
                res_tick[f'{tick}'] = [result_trend, date_list] 
            
        except KeyError:
            break
        res_d[bk] = res_tick